In [3]:
from planet4 import markings, dbscan, io, fnotching

In [101]:
from nbtools.logging import setup_live_logging
import logging

In [102]:
setup_live_logging('planet4', logging.DEBUG)

<Logger planet4 (DEBUG)>

# Data to cluster

In [4]:
from planet4 import region_data, io

In [5]:
db = io.DBManager()

In [4]:
obsids = db.image_names

In [5]:
db.dbname

'/Users/klay6683/Dropbox/data/planet4/2017-12-31_planet_four_classifications_queryable_cleaned_seasons2and3.h5'

In [6]:
len(obsids)

221

# Support functions

In [9]:
from planet4.catalog_production import ReleaseManager
from planet4 import fnotching

In [10]:
rm = ReleaseManager("v1.0")

In [11]:
rm.calc_metadata()

In [5]:
meta = pd.read_csv(rm.metadata_path).set_index('OBSERVATION_ID')

In [12]:
rm.EDRINDEX_meta_path

PosixPath('/Users/klay6683/Dropbox/data/planet4/P4_catalog_v1.0/P4_catalog_v1.0_EDRINDEX_metadata.csv')

In [6]:
meta.loc['ESP_012251_0935']

IMAGE_CENTER_LATITUDE                -86.3968
IMAGE_CENTER_LONGITUDE                99.1505
SOLAR_LONGITUDE                        223.07
START_TIME                2009-03-08 02:10:48
map_scale                                   1
north_azimuth                         137.978
# of tiles                                360
Name: ESP_012251_0935, dtype: object

In [8]:
edrindex = pd.read_hdf("/Volumes/Data/hirise/EDRCUMINDEX.hdf")
p4_edr = edrindex[edrindex.OBSERVATION_ID.isin(rm.obsids)].query(
    'CCD_NAME=="RED4"').drop_duplicates(subset='OBSERVATION_ID')

In [10]:
p4_edr.set_index('OBSERVATION_ID', inplace=True)

In [11]:
p4_edr.loc['ESP_012251_0935']

VOLUME_ID                                                                    MROHR_0001
FILE_NAME_SPECIFICATION               EDR/ESP/ORB_012200_012299/ESP_012251_0935/ESP_...
INSTRUMENT_HOST_ID                                                                  MRO
INSTRUMENT_ID                                                                    HIRISE
PRODUCT_ID                                                       ESP_012251_0935_RED4_0
PRODUCT_VERSION_ID                                                                    1
HICAL_VERSION                                          3.1.18 | 2009-01-23 | 2008-11-17
TARGET_NAME                                                                        MARS
ORBIT_NUMBER                                                                      12251
MISSION_PHASE_NAME                                               Extended Science Phase
RATIONALE_DESC                                                   South polar monitoring
OBSERVATION_START_TIME          

In [13]:
rm.calc_metadata()

UnboundLocalError: local variable 'metadata' referenced before assignment

In [ ]:
rm.

In [61]:
df.set_index('OBSERVATION_ID').to_csv(rm.EDRINDEX_meta_path, index=True)

In [43]:
cols = ['OBSERVATION_ID', 'IMAGE_CENTER_LATITUDE', 'IMAGE_CENTER_LONGITUDE', 'SOLAR_LONGITUDE', 'START_TIME',
        'north_azimuth', '# of tiles']

In [44]:
df[cols].head()

,OBSERVATION_ID,IMAGE_CENTER_LATITUDE,IMAGE_CENTER_LONGITUDE,SOLAR_LONGITUDE,START_TIME,north_azimuth,# of tiles
0,ESP_011296_0975,-82.1965,225.2530,178.833,2008-12-23 16:15:26,110.600107,91
1,ESP_011341_0980,-81.7969,76.1304,180.809,2008-12-27 04:25:02,110.208923,126
2,ESP_011348_0950,-85.0427,259.0940,181.117,2008-12-27 17:29:17,123.624057,91
3,ESP_011350_0945,-85.2160,181.4150,181.205,2008-12-27 21:14:01,99.672793,126
4,ESP_011351_0945,-85.2157,181.5480,181.249,2008-12-27 23:05:54,127.960688,91


In [8]:
fname = "P4_catalog_v1.0_EDRINDEX_metadata.csv"
df.to_csv(rm.savefolder / fname, index=False)

In [9]:
rm.savefolder

PosixPath('/Users/klay6683/Dropbox/data/planet4/P4_catalog_v1.0')

In [11]:
import itertools
import string 

# Adapt this prefix as a unique identifier of the dataset

def fan_id_generator():
    for newid in itertools.product(string.digits+'abcdef', repeat=6):
        yield 'F' + ''.join(newid)

def blotch_id_generator():
    for newid in itertools.product(string.digits+'abcdef', repeat=6):
        yield 'B' + ''.join(newid)

In [22]:
def add_marking_ids(path, fan_id, blotch_id):
    """Add marking_ids for catalog to cluster results.
    
    Parameters
    ----------
    path : str, pathlib.Path
        Path to L1A image_id clustering result directory
    fan_id, blotch_id : generator
        Generator for marking_id
    """
    image_id = path.parent.name
    for kind, id_ in zip(['fans', 'blotches'], [fan_id, blotch_id]):
        fname = str(path / f"{image_id}_L1A_{kind}.csv")
        try:
            df = pd.read_csv(fname)
        except FileNotFoundError:
            continue
        else:
            marking_ids = []
            for i in range(df.shape[0]):
                marking_ids.append(next(id_))
            df['marking_id'] = marking_ids
            df.to_csv(fname, index=False)

In [12]:
from tqdm import tqdm

In [7]:
obsids = rm.obsids

In [ ]:
rm.launch_catalog_production()

In [45]:
rm.calc_metadata()

In [8]:
rm.calc_marking_coordinates()

100%|██████████| 221/221 [00:07<00:00, 28.10it/s]


In [12]:
rm.merge_all()

In [21]:
fan_id = fan_id_generator()
blotch_id = blotch_id_generator()

In [12]:
from planet4.fnotching import fnotch_image_ids, apply_cut

In [31]:
for obsid in tqdm(obsids):
    pm = io.PathManager(obsid=obsid, datapath=rm.savefolder)
    paths = pm.get_obsid_paths('L1A')
    for path in paths:
        add_marking_ids(path, fan_id, blotch_id)

100%|██████████| 221/221 [02:16<00:00,  1.62it/s]


In [16]:
rm.catalog

'P4_catalog_v1.0b4'

In [32]:
for obsid in tqdm(obsids):
    fnotch_image_ids(obsid, savedir=rm.catalog)
    apply_cut(obsid, savedir=rm.catalog)

100%|██████████| 221/221 [13:06<00:00,  3.56s/it]


In [33]:
from planet4.catalog_production import create_roi_file

In [34]:
create_roi_file(obsids, rm.catalog, rm.catalog)

100%|██████████| 221/221 [01:22<00:00,  2.67it/s]


Created /Users/klay6683/Dropbox/data/planet4/P4_catalog_v1.0b4/P4_catalog_v1.0b4_L1C_cut_0.5_fan.csv.
Created /Users/klay6683/Dropbox/data/planet4/P4_catalog_v1.0b4/P4_catalog_v1.0b4_L1C_cut_0.5_blotch.csv.


In [10]:
rm.get_metadata()

100%|██████████| 221/221 [00:12<00:00, 17.95it/s]


In [69]:
rm.merge_all()

In [67]:
fans = rm.read_fan_file()

In [68]:
fans.columns

Index(['angle', 'distance', 'tile_id', 'image_x', 'image_y', 'marking_id',
       'n_votes', 'obsid', 'spread', 'version', 'vote_ratio', 'x', 'x_angle',
       'y', 'y_angle', 'l_s', 'north_azimuth', 'map_scale',
       'BodyFixedCoordinateX', 'BodyFixedCoordinateY', 'BodyFixedCoordinateZ',
       'PlanetocentricLatitude', 'PlanetographicLatitude',
       'PositiveEast360Longitude'],
      dtype='object')

In [32]:
blotches = rm.read_blotch_file()

In [33]:
blotches.columns

Index(['angle', 'image_id', 'image_name', 'image_x', 'image_y', 'marking_id',
       'n_votes', 'obsid', 'radius_1', 'radius_2', 'vote_ratio', 'x',
       'x_angle', 'y', 'y_angle', 'l_s', 'map_scale', 'north_azimuth'],
      dtype='object')

In [34]:
pd.read_csv(rm.metadata_path).columns

Index(['l_s', 'line_samples', 'lines', 'map_scale', 'north_azimuth', 'obsid'], dtype='object')

In [35]:
fans.vote_ratio.isna().value_counts()

False    159293
Name: vote_ratio, dtype: int64

In [36]:
fans.head()

,angle,distance,image_id,image_name,image_x,image_y,marking_id,n_votes,obsid,spread,version,vote_ratio,x,x_angle,y,y_angle,l_s,map_scale,north_azimuth
0,205.56,179.71,APF0000ci9,ESP_012079_0945,2270.76,24336.16,F039765,35,ESP_012079_0945,88.03,1.0,1.00,790.76,-0.90,224.16,-0.43,214.784691,0.25,126.856883
1,185.39,179.62,APF0000cia,ESP_012079_0945,3391.21,5640.60,F039766,15,ESP_012079_0945,21.35,1.0,1.00,431.21,-1.00,160.60,-0.09,214.784691,0.25,126.856883
2,184.98,500.27,APF0000cia,ESP_012079_0945,3509.96,5876.70,F039767,10,ESP_012079_0945,18.91,1.0,1.00,549.96,-1.00,396.70,-0.09,214.784691,0.25,126.856883
3,184.29,105.43,APF0000cia,ESP_012079_0945,3716.27,5824.50,F039769,6,ESP_012079_0945,26.41,1.0,0.68,756.27,-1.00,344.50,-0.07,214.784691,0.25,126.856883
4,189.42,109.50,APF0000cia,ESP_012079_0945,3452.17,6033.00,F03976a,3,ESP_012079_0945,22.58,1.0,0.51,492.17,-0.99,553.00,-0.16,214.784691,0.25,126.856883


In [25]:
rm.launch_catalog_production()

IntProgress(value=0, max=220)

100%|██████████| 221/221 [01:30<00:00,  2.45it/s]


Created /Users/klay6683/Dropbox/data/planet4/P4_catalog_v1.0b4/P4_catalog_v1.0b4_L1C_cut_0.5_fan.csv.
Created /Users/klay6683/Dropbox/data/planet4/P4_catalog_v1.0b4/P4_catalog_v1.0b4_L1C_cut_0.5_blotch.csv.


IntProgress(value=0, max=220)

100%|██████████| 221/221 [00:11<00:00, 18.47it/s]


# Debugging

In [ ]:
from planet4.fnotching import get_clusters_in_path

def find_bad_obsid(obsid, savedir):
    pm = io.PathManager(obsid=obsid, datapath=savedir)
    paths = pm.get_obsid_paths('L1A')
    if len(paths) == 0:
        logger.warning("No paths to fnotch found for %s", obsid)
    fans = []
    blotches = []
    for path in paths:
        f, b = get_clusters_in_path(path)
        fans.append(f)
        blotches.append(b)
    fans = pd.concat(fans, ignore_index=True)

In [24]:
folder

'catalog_1.0b2'

In [ ]:
for obsid in obsids:
    print(obsid)
    find_bad_obsid(obsid, folder)

In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
from planet4.io import get_subframe

In [ ]:
obsid = 'ESP_022273_0950'

In [ ]:
data = db.get_image_name_markings(obsid)[['image_id', 'image_url']]

In [ ]:
data.set_index('image_id', drop=True, inplace=True)

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
def show_subframe(image_id):
    img = get_subframe(data.loc[data.index[0]].values[0])
    return hv.Raster(img)

In [ ]:
%output size=150

In [ ]:
%%opts Raster [xaxis=None, yaxis=None]
show_subframe(data.index[0])+show_subframe(data.index[1])

In [ ]:
dmap = hv.DynamicMap(show_subframe, kdims=['image_id'])
dmap

In [ ]:
dmap.redim.values(image_id=data.index.values)

In [ ]:
xvals = np.linspace(-4,0,202)
yvals = np.linspace(4,0,202)
xs,ys = np.meshgrid(xvals, yvals)

def waves_image(alpha, beta):
    return hv.Image(np.sin(((ys/alpha)**alpha+beta)*xs))

waves_image(0,0) + waves_image(0,4)

In [ ]:
dmap = hv.DynamicMap(waves_image, kdims=['alpha', 'beta'])
dmap

In [ ]:
dmap[0,1] + dmap.select(alpha=1, beta=2)

In [ ]:
dmap.redim.range(alpha=(0,5.0), beta=(1,5.0))

In [ ]:
hv.__version__

In [ ]:
for imgid in image_ids:
    do_cluster_ids(imgid, "without_large_fan_fnotch_on_p4_coords" )

In [ ]:
obsid = 'ESP_020930_0980'
savedir = 'fnotching_per_obsid'

In [ ]:
dbscanner = dbscan.DBScanner(savedir=savedir)

In [ ]:
dbscanner.cluster_image_name(obsid)

In [ ]:
pm.final_blotchfile

In [ ]:
fnotching.fnotch_obsid(obsid, savedir=savedir)

In [ ]:
fnotching.apply_cut_obsid(obsid, savedir=savedir)

In [ ]:
data = region_data.Inca.season2 + region_data.Inca.season3

In [ ]:
do_clustering(ithaca[0])

In [ ]:
ithaca[0]

In [ ]:
from nbtools import execute_in_parallel

In [ ]:
result = execute_in_parallel(do_clustering, data)

# Creating obsid files

In [96]:
from planet4.fnotching import get_clusters_in_path

In [99]:
pm = io.PathManager(obsid=roi.seasons[0], datapath=folder)

NameError: name 'roi' is not defined

In [98]:
pm.obsid_final_blotches_path

PosixPath('/Users/klay6683/Dropbox/data/planet4/catalog_1.0b2/ESP_011408_0930_blotches.csv')

In [21]:
from planet4.catalog_production import create_roi_file

In [22]:
savedir

'catalog_1.0b3'

In [24]:
create_roi_file(obsids, 'catalog', savedir)

100%|██████████| 221/221 [02:56<00:00,  1.25it/s]


In [118]:
pm = io.PathManager(obsid=obsids[0], datapath=savedir)

In [119]:
folders = pm.get_obsid_paths('L1C')

In [132]:
from planet4.catalog_production import read_csvfiles_into_lists_of_frames, concat_bucket_lists

In [ ]:
bucket.items()

In [102]:
roi.available_seasons

['season2', 'season3']

In [125]:
folder

'catalog_1.0b2'

In [104]:
def create_roi_file_with_seasons(roi, datapath, copy_to_analysis=True):
    Fans = []
    Blotches = []
    for season in roi.available_seasons:
        print(season)
        # keep lower case marking objects to add season column later
        fans = []
        blotches = []
        for obsid in getattr(roi, season):
            print(obsid)
            pm = io.PathManager(obsid=obsid, datapath=datapath)
            fans.append(pm.final_fandf)
            blotches.append(pm.final_blotchdf)

        fans = pd.concat(fans, ignore_index=True)
        fans['season'] = season
        Fans.append(fans)
    
        blotches = pd.concat(blotches, ignore_index=True)
        blotches['season'] = season
        Blotches.append(blotches)
        
    Fans = pd.concat(Fans, ignore_index=True)
    Blotches = pd.concat(Blotches, ignore_index=True)
    savedir = pm.path_so_far.parent
    for marks in ['Fans', 'Blotches']:
        savename = f"{roi.name}_{pm.L1C_folder}_{marks}.csv"
        savepath = savedir / savename
        obj = eval(marks)  # get handle to the fan or blotch object
        obj = obj.dropna(how='all', axis=1)
        obj.to_csv(savepath, index=False)
        if copy_to_analysis is True:
            savefolder = io.analysis_folder() / folder / f"{roi.name.lower()}"
            savefolder.mkdir(exist_ok=True)
            savepath = savefolder / savename
            obj.to_csv(savepath, index=False)

In [132]:
create_roi_file_with_seasons(roi, folder)

season2
ESP_011460_0980
ESP_011526_0980
ESP_011737_0980
ESP_012515_0980
ESP_012594_0980
ESP_012805_0980
ESP_012871_0980
season3
ESP_022510_0980
ESP_021587_0980
ESP_021574_0980
ESP_021521_0980
ESP_021508_0980
ESP_020941_0980
ESP_020875_0980
ESP_020374_0980
ESP_020163_0980


In [ ]:
def create_obsid_result_files(obsid, datapath=None):
#     logger.debug("Working on %s", obsid)
    pm = io.PathManager(obsid=obsid, datapath=datapath)
    # the pm.get_obsid_paths method searches for existing image_id folders inside each
    # obsid folder
    folders = list(pm.get_obsid_paths(pm.L1C_folder))
    blotches = []
    fans = []
    for folder in folders:
        pm.id = fnotching.get_id_from_path(folder)
        if pm.final_blotchfile.exists() and pm.final_blotchfile.lstat().st_size > 1:
            blotches.append(pm.final_blotchdf)
        if pm.final_fanfile.exists() and pm.final_fanfile.lstat().st_size > 1:
            fans.append(pm.final_fandf)

    if len(blotches) > 0:
        blotches = pd.concat(blotches, ignore_index=True)
        blotches.dropna(how='all', axis=1, inplace=True)
        blotches.to_csv(pm.obsid_final_blotches_path, index=False)
        print("Created", pm.obsid_final_blotches_path)

    if len(fans) > 0:
        fans = pd.concat(fans, ignore_index=True)
        fans.dropna(how='all', axis=1, inplace=True)
        fans.to_csv(pm.obsid_final_fans_path, index=False)
        print("Created", pm.obsid_final_fans_path)

In [ ]:
for obsid in data:
    create_obsid_result_files(obsid, datapath='for_chase')

In [ ]:
region = getattr(region_data, name)
for season_no in [2, 3]:
    season = str(season_no)
    bucket = []
    for obsid in getattr(region, f"season{season}"):
        pm = io.PathManager(obsid=obsid)
        df = pd.read_csv(pm.obsid_final_blotches_path)
        df['obsid'] = obsid
        bucket.append(df)

    df = pd.concat(bucket, ignore_index=True)
    df.to_csv(io.analysis_folder() / f"p4_catalog/{name}_season{season}_blotches.csv",
              index=False)

In [ ]:
store_folder = io.analysis_folder() / 'for_chase'
store_folder.mkdir(exist_ok=True)

In [ ]:
name = 'inca'

In [ ]:
p = Path('/Users/klay6683/Dropbox/data/planet4/for_chase/' + name)
for kind in ['blotches', 'fans']:
    bucket = []
    filepaths = p.glob(f'*_{kind}.csv')
    for filepath in filepaths:
        obsid = filepath.name[:15]
        df = pd.read_csv(filepath)
        df['obsid'] = obsid
        bucket.append(df)
    df = pd.concat(bucket, ignore_index=True)
    savedir = store_folder / f'{name}'
    savedir.mkdir(exist_ok=True)
    savepath = savedir / f"{name}_{kind}.csv"
    print(savepath)
    df.to_csv(savepath, index=False)
        